In [ ]:
import pathlib
import zipfile
import io
from PIL import Image
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Dataset    

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path, transform=None) -> None:
        super().__init__()
        self.path = pathlib.Path(path)
        assert self.path.exists(), f"{path} does not exist"
        self.transform = transform
        self.files_path = self._get_file_list()
    
    def _get_file_list(self):
        files_path = []
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_file:
            for name in zip_file.namelist():
                if '/' in name:
                    files_path.append(name)
        return files_path

    def __len__(self): 
        return len(self.file_list)
    
    def __getitem__(self, index):
        image, label = None, None
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_file:
            with zip_file.open(self.files_path[index]) as file:
                image = Image.open(file).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            
            label = int(self.files_path[index].split('/')[0])
        return image, label

In [ ]:
class CustomModel(nn.Module): #? Custom model made from scratch
    def __init__(self) -> None:
        super().__init__()
        pass

    def forward(self, x): pass

In [ ]:
class VGG(nn.Module):
    def __init__(self, num_classes=1000) -> None:
        super().__init__()
        self.model = torchvision.models.vgg16(pretrained=True)
        self.model.classifier[6] = nn.Linear(4096, num_classes)

    def forward(self, x):
        pass

class ResNet(nn.Module):
    def __init__(self, num_classes=1000) -> None:
        super().__init__()
        self.model = torchvision.models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        pass